# Implement an NLG chatbot
This file provides easy implementation of a Hugging Face model and fine-tunes it on a custom dataset.  
Simply run the cells to train and chat with a model.  
The cells use scripts created by Hugging Face and each script has many customizable parameters. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Engineering/Curriculum/8th Semester/Final Year Project/cs-nlg-fastapi'

/content/drive/.shortcut-targets-by-id/1bErAQLMRG4YDibru-KHUpGdLMIdUvKef/cs-nlg-fastapi


In [ ]:
!ls 

conversations.txt		      hugging-face	  runs
cs-nlgbot-fastapi.ipynb		      lastTweet.txt	  training_schema.json
cs_training_data.json		      make_dataset.ipynb
dataset_cache_OpenAIGPTTokenizer.zip  requirements.txt


In [ ]:
# Install the /cs-nlg/requirements.txt and upload the resources for spacy

!pip install -r ./requirements.txt
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 259 kB 13.6 MB/s 
     |████████████████████████████████| 499 kB 58.9 MB/s 
     |████████████████████████████████| 216 kB 76.5 MB/s 
     |████████████████████████████████| 880 kB 62.0 MB/s 
     |████████████████████████████████| 132 kB 73.5 MB/s 
     |████████████████████████████████| 5.6 MB 25.2 MB/s 
     |████████████████████████████████| 1.2 MB 58.4 MB/s 
     |████████████████████████████████| 79 kB 11.9 MB/s 
     |████████████████████████████████| 9.0 MB 57.3 MB/s 
     |████████████████████████████████| 127 kB 67.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b9cd458c9620e6dae1db6792e2a8ddb557e4b447fea426b664b2d6d29b24c11b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: u

## Train the model
The train.py script imports a pretrained BERT model and trains it on a given dataset or the default dataset in Hugging Face's S3 bucket.   
With the parameters configured in the following cell, the training should take about 5 minutes using a GPU or around 15 minutes with a cpu.    

Below is a table of the available parameters that can be added to the run command.  

Argument | Type | Default value | Description
---------|------|---------------|------------
dataset_path | `str` | `""` | Path or url of the dataset. If empty download from S3.
dataset_cache | `str` | `'./dataset_cache.bin'` | Path or url of the dataset cache
model | `str` | `"openai-gpt"` | Path, url or short name of the model
num_candidates | `int` | `2` | Number of candidates for training
max_history | `int` | `2` | Number of previous exchanges to keep in history
train_batch_size | `int` | `4` | Batch size for training
valid_batch_size | `int` | `4` | Batch size for validation
gradient_accumulation_steps | `int` | `8` | Accumulate gradients on several steps
lr | `float` | `6.25e-5` | Learning rate
lm_coef | `float` | `1.0` | LM loss coefficient
mc_coef | `float` | `1.0` | Multiple-choice loss coefficient
max_norm | `float` | `1.0` | Clipping gradient norm
n_epochs | `int` | `3` | Number of training epochs
personality_permutations | `int` | `1` | Number of permutations of personality sentences
device | `str` | `"cuda" if torch.cuda.is_available() else "cpu"` | Device (cuda or cpu)
fp16 | `str` | `""` | Set to O0, O1, O2 or O3 for fp16 training (see apex documentation)
local_rank | `int` | `-1` | Local rank for distributed training (-1: not distributed)


In [ ]:
# Run the train.py script created by Hugging Face to fine-tune a BERT model on cs_training_data.json. If the dataset_path is removed, the script will automatically upload the dataset hosted by Hugging Face.

# !python ./hugging-face/train.py --dataset_path="cs_training_data.json" --n_epochs=200 --train_batch_size=4 --valid_batch_size=2 --max_history=4

## Implement the model  
Chat with the trained model using interact.py.    
If a model_checkpoint is not specified, interact.py will reference a previously trained model from Hugging Face's S3 bucket.  

Argument | Type | Default value | Description
---------|------|---------------|------------
dataset_path | `str` | `""` | Path or url of the dataset. If empty download from S3.
dataset_cache | `str` | `'./dataset_cache.bin'` | Path or url of the dataset cache
model | `str` | `"openai-gpt"` | Path, url or short name of the model
max_history | `int` | `2` | Number of previous utterances to keep in history
device | `str` | `cuda` if `torch.cuda.is_available()` else `cpu` | Device (cuda or cpu)
no_sample | action `store_true` | Set to use greedy decoding instead of sampling
max_length | `int` | `20` | Maximum length of the output utterances
min_length | `int` | `1` | Minimum length of the output utterances
seed | `int` | `42` | Seed
temperature | `int` | `0.7` | Sampling softmax temperature
top_k | `int` | `0` | Filter top-k tokens before sampling (`<=0`: no filtering)
top_p | `float` | `0.9` | Nucleus filtering (top-p) before sampling (`<=0.0`: no filtering)


In [ ]:
# Example location from a Colab run
#!python ./CustomerServiceBot-RW/hugging-face/interact.py --model_checkpoint='./runs/Apr03_19-59-03_b8f756943518_openai-gpt'

!python ./hugging-face/interact.py  --model_checkpoint='./runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt' --max_length=249

INFO:./hugging-face/interact.py:Namespace(dataset_cache='./dataset_cache', dataset_path='', device='cuda', max_history=2, max_length=249, min_length=1, model='openai-gpt', model_checkpoint='./runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt', no_sample=False, seed=0, temperature=0.7, top_k=0, top_p=0.9)
INFO:./hugging-face/interact.py:Get pretrained model and tokenizer
INFO:transformers.tokenization_utils:Model name './runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt' not found in model shortcut name list (openai-gpt). Assuming './runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:loading file ./runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt/vocab.json
INFO:transformers.tokenization_utils:loading file ./runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt/merges.txt
INFO:transformers.tokenization_utils:loading file ./runs/Jun19_19-12-12_4f93e2530c0a_openai-gpt/added_tokens.json
INFO:transformers.t

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [698]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://c5f4-34-141-178-204.ngrok.io
INFO:     2406:7400:61:81cb:6151:29cf:a879:b697:0 - "GET / HTTP/1.1" 200 OK
INFO:     2406:7400:61:81cb:6151:29cf:a879:b697:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     42.104.120.133:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.207.225.181:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [698]


In [ ]:
# ! nvidia-smi

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!which nvcc

In [ ]:
# !cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

In [ ]:
# !python3 -c 'print(__import__("tensorflow").__version__)'

In [ ]:
# !python

In [ ]:
!pip freeze

absl-py==1.1.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.2
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.7.7
bokeh==2.3.3
boto3==1.24.22
botocore==1.27.22
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.7
certifi==2022.6.15
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.5
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descar